In [1]:
%config IPCompleter.greedy = True
%config InlineBackend.figure_format = 'retina'
%matplotlib inline
%load_ext tensorboard

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sn
import tensorflow as tf
from datetime import datetime

pd.set_option('mode.chained_assignment', None)
sn.set(rc={'figure.figsize':(9,9)})
sn.set(font_scale=1.4)

# make results reproducible
seed = 0
np.random.seed(seed)

!pip install pydot
!rm -rf ./logs/ 

# Estimators

TensorFlow Estimators are a high-level representation of a complete model, which has been designed for easy scaling and asynchronous training. However it is recommended to use the Keras API instead, which has greater flexibility in creating a model, however we include estimators for completeness.

An estimator encapsulates training, evaluation, prediction and export for serving. They have the benefit that the can easily be run on a local host machine or on a distributed multi-server enviroment, with CPU's, GPU's or TPU's without changing the model. They provide a safe distributed training loop that controls loading data, handling exceptions, creating model checkpoints to recover from failures and saving summaries for TensorBoard. When we work with estimators, we must seperate the data input pipeline from the model.

## Pre-made Estimators

TensorFlow broadly supports the following estimators (from `tf.estimator`'s) for Classification
* `BoostedTreesClassifier` : Boosted Trees model
* `LinearClassifier` : Linear model
* `DNNClassifier` : Deep neural network model
* `DNNLinearCombinedClassifier` : Deep neural network and linear joined model 

And similary for regression
* `BoostedTreesRegressor` : Boosted Trees model
* `LinearRegressor` : Linear model
* `DNNRegressor` : Deep neural network model
* `DNNLinearCombinedRegressor` : Deep neural network and linear joined model 

## Automatic data pre-processing

One of the benefits that Estimators have is that we specify what data type our features are and it will correctly pre-process the features for that model, i.e `numeric`, `categorical` or `string` based data. As with Keras we would have to pre-process our features before passing them to the model. We often want to turn `categorical` features into an encoding as *one-hot* representation and use an embedding to map `strings` either categorical representation (*one-hot* encodings) or some other representation such as numeric vector.

[Some](https://www.tensorflow.org/api_docs/python/tf/feature_column) of the datatypes, `tf.feature_column` that we can specify are:
* `numeric_column` : numeric features
* `categorical_column_with_vocabulary_list` : `CategoricalColumn` with a vocabulary list
* `embedding_column` : converts sparse input to categorical input
* `bucketized_column` : discretized dense input bucketed by `boundaries`

## Setup

To use estimators we have to pass a dataset loading function during training, evaluation and predicting. This function must return a tuple of two objects:
* Dictionary, of keys of *feature_name*'s, with corresponding values of *feature_tensor*'s
* Targets, tensor

For example on our digits dataset for classification

In [2]:
# Loading our digits dataset
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
digits = datasets.load_digits()

(X, y) = datasets.load_digits(return_X_y=True)
X = X.astype(np.float32)
y = y.astype(np.int32)

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42)

# Pre process the data
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Create the datset loading functions
def input_train_fn():
    return {'X': X_train}, y_train

def input_test_fn():
    return {'X': X_test}, y_test

We have to specify to the estimator what type of data is each feature is using the `tf.feature_column` types. In this example we have numeric data

In [3]:
feature_columns = [tf.feature_column.numeric_column('X', shape=[64])]

We can instantiate the estimator, here we will use the `tf.estimator.DNNClassifier` for our classification problem.


For the `tf.estimator.DNNClassifier` [some](https://www.tensorflow.org/api_docs/python/tf/estimator/DNNClassifier) of the keyword arguments it takes are:
* `hidden_units` : List of number of hidden units for each fully connected layer
* `feature_columns` : List of all of the `tf.feature_column` types for the input features
* `n_classes` : Number of classes
* `optimizer` : Instance of `tf.keras.optimizer.*` used to train the model, can also be a built-in string
* `activation_function` : Activation function to use for each layer, defaults to `tf.nn.relu`
* `dropuout` : Probability to drop a given neuron during training, defaults to None
* `batch_norm` : Whether to use batch normalization after each hidden layer

In [4]:
# Specify how many layers and what hidden units they have are
hidden_units = [64, 64, 64, 10]

# Number of classes
n_classes = 10

# Create the estimator
classifier = tf.estimator.DNNClassifier(
    feature_columns=feature_columns,
    hidden_units=hidden_units,
    n_classes=n_classes,
    optimizer='Adam',
    dropout=0.1,
    batch_norm=True)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/var/folders/mg/rx49509d49q70jr6f72nnrbc0000gn/T/tmppbpzul8w', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': ClusterSpec({}), '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


We can then train our model passing the function to load the training dataset in the expected format. Note here we don't have epochs instead we have steps, which is how many steps to train the model for. Here we can approximate epochs with the following

In [6]:
epochs = 10
steps = epochs * X_train.shape[0]

classifier.train(input_fn=input_train_fn,
                steps=steps)

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Instructions for updating:
Use Variable.read_value. Variables in 2.X are initialized automatically both in eager and graph (inside tf.defun) contexts.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 0 into /var/folders/mg/rx49509d49q70jr6f72nnrbc0000gn/T/tmppbpzul8w/model.ckpt.
INFO:tensorflow:loss = 2.7918434, step = 0
INFO:tensorflow:global_step/sec: 168.633
INFO:tensorflow:loss = 0.6459314, step = 100 (0.594 sec)
INFO:tensorflow:global_step/sec: 233.872
INFO:tensorflow:loss = 0.3376872, step = 200 (0.428 sec)
INFO:tensorflow:global_step/sec: 218.224
INFO:tensorflow:loss = 0.1794372, step = 300 (0.458 sec)
INFO:tensorflow:global_step/sec: 234.035
INFO:tensorflow:loss = 0.13259257,

INFO:tensorflow:global_step/sec: 194.177
INFO:tensorflow:loss = 0.010542458, step = 7100 (0.515 sec)
INFO:tensorflow:global_step/sec: 185.659
INFO:tensorflow:loss = 0.0074856337, step = 7200 (0.539 sec)
INFO:tensorflow:global_step/sec: 188.539
INFO:tensorflow:loss = 0.004575637, step = 7300 (0.530 sec)
INFO:tensorflow:global_step/sec: 218.776
INFO:tensorflow:loss = 0.0070897806, step = 7400 (0.457 sec)
INFO:tensorflow:global_step/sec: 227.375
INFO:tensorflow:loss = 0.012254769, step = 7500 (0.440 sec)
INFO:tensorflow:global_step/sec: 240.826
INFO:tensorflow:loss = 0.0057765017, step = 7600 (0.415 sec)
INFO:tensorflow:global_step/sec: 247.145
INFO:tensorflow:loss = 0.00534432, step = 7700 (0.405 sec)
INFO:tensorflow:global_step/sec: 247.886
INFO:tensorflow:loss = 0.0084147565, step = 7800 (0.403 sec)
INFO:tensorflow:global_step/sec: 230.417
INFO:tensorflow:loss = 0.0042782207, step = 7900 (0.434 sec)
INFO:tensorflow:global_step/sec: 246.015
INFO:tensorflow:loss = 0.008331404, step = 800

Can then pass a function to load our test dataset, and evaluate our model, averaging the evaluation metrics

In [7]:
eval_result = classifier.evaluate(input_fn=input_test_fn, steps=100)
print(eval_result)
print('\nTest accuracy: {:.3%}'.format(eval_result['accuracy']))

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2020-04-26T19:24:16Z
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /var/folders/mg/rx49509d49q70jr6f72nnrbc0000gn/T/tmppbpzul8w/model.ckpt-14370
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Evaluation [10/100]
INFO:tensorflow:Evaluation [20/100]
INFO:tensorflow:Evaluation [30/100]
INFO:tensorflow:Evaluation [40/100]
INFO:tensorflow:Evaluation [50/100]
INFO:tensorflow:Evaluation [60/100]
INFO:tensorflow:Evaluation [70/100]
INFO:tensorflow:Evaluation [80/100]
INFO:tensorflow:Evaluation [90/100]
INFO:tensorflow:Evaluation [100/100]
INFO:tensorflow:Inference Time : 0.39889s
INFO:tensorflow:Finished evaluation at 2020-04-26-19:24:17
INFO:tensorflow:Saving dict for global step 14370: accuracy = 0.975, average_loss = 0.5407167, global_step = 14370, loss = 0.54071695
INFO:tensorflow:Saving 'checkpo

We can see we have somewhat easily trained a simple classification model with a sufficient test accuracy.

Can easily make predictions using our trained model as well using the `tf.estimator.DNNClassifier.predict()`



# TensorBoard